# Final submission

- less gates, less noises: optimze a shallow circuit with naive VQE + CHC-S + COBYLA
- unbalanced better than balanced: a bit random disturb on ansatz `zeros` init point
- less pulses, less noises and faster: apply instrution optimization at QASM level

In [1]:
from pprint import pprint as pp
import utils; utils.log = lambda *args, **kwargs: None   # mute debug log
from run_eval import *

# the common settings for all
args = Namespace()
args.S = 'Montreal'           # system model
args.N = ''                   # noise model
args.simulator = 'automatic'  # Aer simulator
args.device = 'CPU'           # Aer device
args.shots = 6000             # fixed as contest
args.H = str(HAM_FILE)        # load ham from 'OHhamiltonian.txt'
args.thresh = 0.0             # ham trim thresh
args.exp_dp = None            # leave it blank so far
args.seed = None              # leave it blank so far
print('args:')
pp(vars(args))
print()

# the required seeds to average
print('required seeds:', seeds)

args:
{'H': 'd:\\Desktop\\Workspace\\@Contest\\ICCAD 2023 Drug Discovery '
      'Challenge\\QC-Contest-Demo\\Hamiltonian\\OHhamiltonian.txt',
 'N': '',
 'S': 'Montreal',
 'device': 'CPU',
 'exp_dp': None,
 'seed': None,
 'shots': 6000,
 'simulator': 'automatic',
 'thresh': 0.0}

required seeds: [20, 21, 30, 33, 36, 42, 43, 55, 67, 170]


⚪ ref_val

In [2]:
# the ref_val (ground truth)
ref_val = solver_const(None, None)
print('ref_val:', ref_val)

ref_val: -74.3871462681872


In [3]:
# helper func
def eval_with_seed(args, seed:int, run_pulse:bool=True) -> Tuple[float, int]:
  args.seed = seed
  seed_everything(seed)
  return eval(args, is_run_pulse=run_pulse)

mean = lambda x: sum(x) / len(x) if len(x) else 0.0
error = lambda x, y: abs(x - y)
error_rate = lambda x, y: error(x, y) / abs(y)

def eval_qsam_over_seeds(args):
  # load pretrained qasm
  args.qsam_fp = args.exp_dp / 'ansatz_t.qsam'
  args.layout_fp = args.exp_dp / 'ansatz_t.layout'

  # run over different seeds
  energies, duration = [], None
  for i, seed in enumerate(seeds, 1):
    ene, dur = eval_with_seed(args, seed, run_pulse=i==1)
    print(f'[{i}/{len(seeds)}] seed: {seed}, ene: {ene}')
    energies.append(ene)
    if dur is not None: duration = dur
  enegry = mean(energies)

  # average results
  print('=' * 42)
  print('enegry:', enegry)
  print('duration:', duration)
  print('error:', error(enegry, ref_val))
  print(f'error rate: {error_rate(enegry, ref_val):%}')

⚪ cairo

In [4]:
args.N = 'cairo'
args.exp_dp = Path('log') / 'final_cairo'

eval_qsam_over_seeds(args)

[1/10] seed: 20, ene: -69.05218947681885
[2/10] seed: 21, ene: -68.89457546446678
[3/10] seed: 30, ene: -69.02147648629943
[4/10] seed: 33, ene: -68.85758946315644
[5/10] seed: 36, ene: -69.00565872186435
[6/10] seed: 42, ene: -68.700140925467
[7/10] seed: 43, ene: -68.94478015248717
[8/10] seed: 55, ene: -69.01433758465011
[9/10] seed: 67, ene: -68.94170174651723
[10/10] seed: 170, ene: -68.65752906151835
enegry: -68.90899790832457
duration: 54720
error: 5.4781483598626295
error rate: 7.364375%


⚪ kolkata

In [5]:
args.N = 'kolkata'
args.exp_dp = Path('log') / 'final_kolkata'

eval_qsam_over_seeds(args)

[1/10] seed: 20, ene: -70.18289448282869
[2/10] seed: 21, ene: -70.21466700528333
[3/10] seed: 30, ene: -70.2319970631723
[4/10] seed: 33, ene: -70.14396770085001
[5/10] seed: 36, ene: -70.08224031572259
[6/10] seed: 42, ene: -69.8645170611019
[7/10] seed: 43, ene: -70.07517777918355
[8/10] seed: 55, ene: -70.14602410379399
[9/10] seed: 67, ene: -70.09435919182089
[10/10] seed: 170, ene: -70.11057439152863
enegry: -70.11464190952859
duration: 54720
error: 4.272504358658608
error rate: 5.743606%


⚪ montreal

In [6]:
args.N = 'montreal'
args.exp_dp = Path('log') / 'final_montreal'

eval_qsam_over_seeds(args)

[1/10] seed: 20, ene: -68.8865752186636
[2/10] seed: 21, ene: -69.09429634611166
[3/10] seed: 30, ene: -68.99392900601022
[4/10] seed: 33, ene: -68.83621523897223
[5/10] seed: 36, ene: -68.89169012827557
[6/10] seed: 42, ene: -68.88937660244358
[7/10] seed: 43, ene: -68.64393270047646
[8/10] seed: 55, ene: -68.9878615241781
[9/10] seed: 67, ene: -68.92158633321674
[10/10] seed: 170, ene: -69.03280376221738
enegry: -68.91782668605656
duration: 54720
error: 5.4693195821306375
error rate: 7.352506%


⚠ noiseless (for debug use, not for final submission)

In [ ]:
args.N = ''
args.exp_dp = Path('log') / 'test'

eval_qsam_over_seeds(args)